In [0]:
import pandas as pd 
import numpy as np
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from sklearn.preprocessing import  LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

Using TensorFlow backend.


Select your training and testing files below.

In [0]:
#### upload traind and test file
from google.colab import files
uploaded = files.upload()


Saving training.csv to training.csv


In [0]:
uploaded = files.upload()


Saving testing.csv to testing.csv


In [0]:
train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('testing.csv')

In [0]:
train_df.head(4)

,Category,Date_and_Time,Keywords,Article
0,Entertainment,"\nAugust 23, 2018 16:58 IST\n",\nBring on the laughs,"A dash of stand up comedy, lots of improvisat..."
1,Entertainment,"\nSeptember 08, 2018 11:53 IST\n",\nTamannaah Bhatia to play leading lady in Sun...,Actor Tamannaah Bhatia is excited to have sig...
2,Entertainment,"\nOctober 26, 2018 12:57 IST\n",\nSensation Rise 2018: Rise for the city and f...,They say you never forget your first. Tri-city...
3,Entertainment,"\nSeptember 18, 2018 19:35 IST\n",\nThe Beatles in India\n,It was in the year 1968 when the Beatles were...


In [0]:
train_df = train_df[~train_df.Article.isnull()]
test_df =  test_df[~test_df.Article.isnull()]

In [0]:
le=LabelEncoder()
le.fit(train_df.Category.values)

LabelEncoder()

In [0]:
train_df=train_df.sample(frac=1)

In [0]:
X_train = train_df[['Article']]
X_test =  test_df[['Article']]
y_train = train_df['Category']
y_test = test_df['Category']

We will do **text preprocessing** for **BERT**

In [0]:
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [0]:
train_sentences = X_train.Article.values
test_sentences= X_test.Article.values

In [0]:
train_sentences = train_sentences.tolist()
test_sentences = test_sentences.tolist()

In [0]:
len(train_sentences),y_train.shape

(1430, (1430,))

In [0]:
train_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in train_sentences]
test_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in test_sentences]

In [0]:
!pip install pytorch_transformers
from pytorch_transformers import *

     |████████████████████████████████| 184kB 1.4MB/s 
     |████████████████████████████████| 870kB 64.1MB/s 
     |████████████████████████████████| 1.0MB 62.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e4ed3fc7273b45d0007d31985473eb91619e2605932ceb05323c72fe69565d3e
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
%%time
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
train_tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
test_tokenized_texts = [tokenizer.tokenize(sent) for sent in test_sentences]
print ("Tokenize the first sentence:")
print (train_tokenized_texts[0])

100%|██████████| 213450/213450 [00:00<00:00, 401818.53B/s]


Tokenize the first sentence:
['[CLS]', '>', '>', 'Some', 'er', '##rone', '##ous', 'figures', 'were', 'cited', 'in', 'the', 'editorial', 'on', 'man', '##go', 'exports', ',', '“', '>', 'So', '##ur', 'turn', 'in', 'India', '-', 'EU', 'trade', '”', '(', 'May', '2', ',', '2014', ')', '.', 'Editorial', 'writer', '’', 's', 'c', '##lar', '##ification', ':', '“', 'Total', 'Indian', 'exports', 'to', 'EU', 'countries', 'in', '2012', '-', '13', 'was', 'only', '4', ',', '07', '##1', 'tonnes', '.', 'The', 'figure', 'of', '16', 'million', 'mentioned', 'was', 'not', 'in', 'terms', 'of', 'tonnes', 'but', 'absolute', 'number', 'of', 'man', '##go', '##es', 'exported', 'to', 'the', 'United', 'Kingdom', 'alone', '.', 'Total', 'exports', 'of', 'man', '##go', '##es', 'from', 'India', 'in', '2012', '-', '13', 'added', 'up', 'to', 'only', '55', ',', '58', '##4', '.', '98', 'tonnes', '(', 'according', 'to', 'data', 'from', 'the', 'Agricultural', 'and', 'Process', '##ed', 'Food', 'Products', 'Expo', '##rt', 'Dev

In [0]:
train_input_ids=[]
for i in tqdm_notebook(range(len(train_tokenized_texts))):
  train_input_ids.append(tokenizer.convert_tokens_to_ids(train_tokenized_texts[i]))


Token indices sequence length is longer than the specified maximum sequence length for this model (975 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1175 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1344 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1028 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (807 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

In [0]:
test_input_ids=[]
for i in tqdm_notebook(range(len(test_tokenized_texts))):
  test_input_ids.append(tokenizer.convert_tokens_to_ids(test_tokenized_texts[i]))


Token indices sequence length is longer than the specified maximum sequence length for this model (1541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1271 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (895 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (928 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1553 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

In [0]:
##Padding 
MAX_LEN = 512
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
#Create attention masks
train_attention_masks = []
test_attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in train_input_ids:
  seq_mask = [float(i>0) for i in seq]
  train_attention_masks.append(seq_mask)
  
# Create a mask of 1s for each token followed by 0s for padding
for seq in test_input_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

In [0]:
train_inputs = torch.tensor(train_input_ids)
validation_inputs = torch.tensor(test_input_ids)
train_labels = torch.tensor(y_train)
validation_labels = torch.tensor(y_test)
train_masks = torch.tensor(train_attention_masks)
validation_masks = torch.tensor(test_attention_masks)

In [0]:
batch_size = 8

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device ='cpu'
model = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=le.classes_.shape[0])
model.to(device)

100%|██████████| 435779157/435779157 [00:10<00:00, 40072056.89B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
lr = 2e-5
max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1


### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler

In [0]:
model_wts='/content/drive/My Drive/News_claasifiers/BERT' ## in our drive wew will make a folder to save model weights

In [0]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []


epochs = 15
val_accuracy=0
# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):
  
  

    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())    
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))
      # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        for i, batch in enumerate(validation_dataloader):
          batch = tuple(t.to(device) for t in batch)
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # print (outputs)
          prediction = torch.argmax(outputs[0],dim=1)
          total += b_labels.size(0)
          correct+=(prediction==b_labels).sum().item()
    curr__val_accuracy=100 * correct / total
    if curr__val_accuracy>val_accuracy:
      val_accuracy=curr__val_accuracy
      torch.save(model.state_dict(), model_wts+'/model_without_language_model.ckpt')
      tokenizer.save_pretrained(model_wts+'/Data')
      print ("Model saved")
    print('Validation accuracy {} {}'.format(epoch,100 * correct / total))  

Epoch [1/15], Step [1/179], Loss: 2.3769
Epoch [1/15], Step [51/179], Loss: 1.3062
Epoch [1/15], Step [101/179], Loss: 0.9204
Epoch [1/15], Step [151/179], Loss: 1.0059
Model saved
Validation accuracy 0 75.35410764872522
Epoch [2/15], Step [1/179], Loss: 0.6115
Epoch [2/15], Step [51/179], Loss: 0.3495
Epoch [2/15], Step [101/179], Loss: 0.6188
Epoch [2/15], Step [151/179], Loss: 0.2436
Model saved
Validation accuracy 1 83.0028328611898
Epoch [3/15], Step [1/179], Loss: 0.2023
Epoch [3/15], Step [51/179], Loss: 0.2409
Epoch [3/15], Step [101/179], Loss: 0.4101
Epoch [3/15], Step [151/179], Loss: 1.0618
Model saved
Validation accuracy 2 84.13597733711048
Epoch [4/15], Step [1/179], Loss: 0.0358
Epoch [4/15], Step [51/179], Loss: 0.3330
Epoch [4/15], Step [101/179], Loss: 0.3334
Epoch [4/15], Step [151/179], Loss: 0.0360
Validation accuracy 3 84.13597733711048
Epoch [5/15], Step [1/179], Loss: 0.0140
Epoch [5/15], Step [51/179], Loss: 0.0352
Epoch [5/15], Step [101/179], Loss: 0.0238
Epo

We will now load the saved model and tokenizer and and give various metrics results

In [0]:
os.listdir(model_wts)

['Data', 'BERT_LARGE', 'model_without_language_model.ckpt']

In [0]:
model = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=le.classes_.shape[0])
model.load_state_dict(torch.load(model_wts+"/model_without_language_model.ckpt"))
model.eval()


100%|██████████| 435779157/435779157 [00:36<00:00, 12062818.10B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
###we will load our transfomed dataset above and get predictions

In [0]:
test_predictions=[]
for i, batch in enumerate(validation_dataloader):
          # Unpack the inputs from our dataloader
          b_input_ids, b_input_mask, b_labels = batch
          # Forward pass
          outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          # print (outputs)
          prediction = torch.argmax(outputs[0],dim=1)
          test_predictions.append(prediction)

In [27]:
print ("done")

done


In [30]:
test_predictions[0].numpy()

array([1, 2, 2, 7, 2, 2, 6, 2])

In [0]:
predictions=[]
for tp in test_predictions:
  predictions.extend(tp.numpy().tolist())

In [0]:
predictions=le.inverse_transform(predictions)

In [0]:
actual_test_labels=le.inverse_transform(y_test)

### Metrics analysis

In [0]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report

In [40]:
##Accuracy 
print ("Accuracy of BERT is {}".format(str(accuracy_score(actual_test_labels,predictions))))

Accuracy of BERT is 0.8498583569405099
ERROR! Session/line number was not unique in database. History logging moved to new session 59


In [41]:
### confusion matrix
pd.crosstab(actual_test_labels, predictions, rownames=['Actual'], colnames=['Predicted'])

Predicted,Business & Economy,Education & Career,Entertainment,Food & Health,International,Others,Politics & Governance,Science & Technology,Sports
Actual,,,,,,,,,
Business & Economy,34,0,0,1,0,2,0,3,0
Education & Career,1,32,1,0,1,3,0,2,0
Entertainment,0,1,36,0,0,0,1,1,0
Food & Health,0,2,0,33,1,1,0,3,0
International,1,0,2,0,36,0,1,0,0
Others,0,2,3,0,3,24,5,3,0
Politics & Governance,1,0,0,0,1,4,28,0,0
Science & Technology,1,0,0,1,1,0,0,37,0
Sports,0,0,0,0,0,0,0,0,40


In [0]:
report = classification_report(actual_test_labels, predictions, output_dict=True)


In [44]:
pd.DataFrame(report).T


,precision,recall,f1-score,support
Business & Economy,0.894737,0.850000,0.871795,40.000000
Education & Career,0.864865,0.800000,0.831169,40.000000
Entertainment,0.857143,0.923077,0.888889,39.000000
Food & Health,0.942857,0.825000,0.880000,40.000000
International,0.837209,0.900000,0.867470,40.000000
Others,0.705882,0.600000,0.648649,40.000000
Politics & Governance,0.800000,0.823529,0.811594,34.000000
Science & Technology,0.755102,0.925000,0.831461,40.000000
Sports,1.000000,1.000000,1.000000,40.000000
accuracy,0.849858,0.849858,0.849858,0.849858
